## CS140 - Group B


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.base import TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 
from sklearn.ensemble import RandomForestClassifier
from plotly.offline import iplot
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split as tts

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [5]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [0]:
TAGS = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
MODALS = ['will', 'would', 'can', 'could', 'may', 'might']
TEMPS = ['before', 'after', 'since']

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/YonglinWang-Brandeis/cs140-groupB-annotation-files/master/code/features_gold.csv')

In [8]:
df.head()


,POS1,POS2,BEFORE_1,AFTER_1,BEFORE_2,AFTER_2,DISTANCE,MODAL,TEMP_BEFORE,TEMP_AFTER,TEMP_SINCE,SYN,DERIVATION,RELATION
0,VBD,VBG,NNP,PRP,VBD,NNS,3,0,0.0,0.0,0.0,0,0,after
1,VBZ,VBD,EX,DT,NNP,PRP,6,0,0.0,0.0,0.0,0,0,vague
2,VBZ,VBD,EX,DT,NNS,NNP,16,0,0.0,0.0,0.0,0,0,vague
3,VBZ,VBN,EX,DT,RB,RBR,42,0,0.0,0.0,0.0,1,0,after
4,VBZ,VBP,EX,DT,PRP,JJR,47,0,0.0,0.0,0.0,0,0,after


In [10]:
df.shape

(662, 14)

In [0]:
def encode_labels(label):
    if label == 'vague':
        return 0
    elif label == 'before':
        return 1
    elif label == 'after':
        return 2
    else:
        return 3

In [0]:
df_copy = df.copy()
df['y'] = df_copy.RELATION.apply(encode_labels)

In [13]:
df.head()

,POS1,POS2,BEFORE_1,AFTER_1,BEFORE_2,AFTER_2,DISTANCE,MODAL,TEMP_BEFORE,TEMP_AFTER,TEMP_SINCE,SYN,DERIVATION,RELATION,y
0,VBD,VBG,NNP,PRP,VBD,NNS,3,0,0.0,0.0,0.0,0,0,after,2
1,VBZ,VBD,EX,DT,NNP,PRP,6,0,0.0,0.0,0.0,0,0,vague,0
2,VBZ,VBD,EX,DT,NNS,NNP,16,0,0.0,0.0,0.0,0,0,vague,0
3,VBZ,VBN,EX,DT,RB,RBR,42,0,0.0,0.0,0.0,1,0,after,2
4,VBZ,VBP,EX,DT,PRP,JJR,47,0,0.0,0.0,0.0,0,0,after,2


In [14]:
print(list(df.columns) )

['POS1', 'POS2', 'BEFORE_1', 'AFTER_1', 'BEFORE_2', 'AFTER_2', 'DISTANCE', 'MODAL', 'TEMP_BEFORE', 'TEMP_AFTER', 'TEMP_SINCE', 'SYN', 'DERIVATION', 'RELATION', 'y']


In [0]:
y_data = df['y']
x_data = df[['POS1', 'POS2', 'BEFORE_1', 'AFTER_1', 'BEFORE_2', 'AFTER_2', 'DISTANCE', 'MODAL', 'TEMP_BEFORE', 'TEMP_AFTER', 'TEMP_SINCE', 'SYN', 'DERIVATION']]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=0)

In [17]:
from collections import Counter
print('dataset shape {}'.format(Counter(y_data)))
print('dataset shape {}'.format(Counter(y_train)))
print('dataset shape {}'.format(Counter(y_test)))

dataset shape Counter({0: 309, 1: 202, 2: 108, 3: 43})
dataset shape Counter({0: 247, 1: 159, 2: 89, 3: 34})
dataset shape Counter({0: 62, 1: 43, 2: 19, 3: 9})


# Naíve Bayes

In [0]:
pipeline = Pipeline([    
    ('vec', OneHotEncoder()) ,
    ('clf', BernoulliNB()),
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [34]:

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=['p_VAGE', 'p_BEFORE', 'p_AFTER','p_SIMU'], index=['VAGUE', 'BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

the accuracy of the model is 0.50

              precision    recall  f1-score   support

           0       0.68      0.58      0.62        73
           1       0.47      0.48      0.47        42
           2       0.21      0.25      0.23        16
           3       0.00      0.00      0.00         2

    accuracy                           0.50       133
   macro avg       0.34      0.33      0.33       133
weighted avg       0.54      0.50      0.52       133


        p_VAGE  p_BEFORE  p_AFTER  p_SIMU
VAGUE       42        13        6       1
BEFORE      16        20        6       1
AFTER        9         6        4       0
SIMU         6         3        0       0


# Logistic Regression

In [0]:
pipeline = Pipeline([    
    ('vec', OneHotEncoder()) ,

  ('clf', LogisticRegression(
      solver='newton-cg',  
      multi_class='multinomial', 
      )),
])


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [0]:

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=['p_VAGE', 'p_BEFORE', 'p_AFTER','p_SIMU'], index=['VAGUE', 'BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

the accuracy of the model is 0.48

              precision    recall  f1-score   support

           0       0.69      0.56      0.62        77
           1       0.40      0.45      0.42        38
           2       0.16      0.20      0.18        15
           3       0.11      0.33      0.17         3

    accuracy                           0.48       133
   macro avg       0.34      0.38      0.35       133
weighted avg       0.53      0.48      0.50       133


        p_VAGE  p_BEFORE  p_AFTER  p_SIMU
VAGUE       43        13        5       1
BEFORE      18        17        7       1
AFTER       12         4        3       0
SIMU         4         4        0       1


# Random Forest

In [0]:
pipeline = Pipeline([    
    ('vec', OneHotEncoder()) ,
  ('clf', RandomForestClassifier(n_estimators = 100,
                        min_samples_split = 2,
                        max_depth = 202,))
])


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [0]:

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=['p_VAGE', 'p_BEFORE', 'p_AFTER','p_SIMU'], index=['VAGUE', 'BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

the accuracy of the model is 0.53

              precision    recall  f1-score   support

           0       0.82      0.54      0.65        94
           1       0.40      0.55      0.46        31
           2       0.11      0.40      0.17         5
           3       0.00      0.00      0.00         3

    accuracy                           0.53       133
   macro avg       0.33      0.37      0.32       133
weighted avg       0.68      0.53      0.58       133


        p_VAGE  p_BEFORE  p_AFTER  p_SIMU
VAGUE       51         7        3       1
BEFORE      24        17        0       2
AFTER       13         4        2       0
SIMU         6         3        0       0


## GradientBoosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [41]:
pipeline = Pipeline([    
  ('vec', OneHotEncoder()) ,
  ('clf', GradientBoostingClassifier())
])

from collections import Counter
print('dataset shape {}'.format(Counter(y_train)))
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=['p_VAGE', 'p_BEFORE', 'p_AFTER','p_SIMU'], index=['VAGUE', 'BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

dataset shape Counter({0: 247, 1: 159, 2: 89, 3: 34})
the accuracy of the model is 0.52

              precision    recall  f1-score   support

           0       0.84      0.53      0.65        98
           1       0.37      0.55      0.44        29
           2       0.05      0.20      0.08         5
           3       0.00      0.00      0.00         1

    accuracy                           0.52       133
   macro avg       0.32      0.32      0.29       133
weighted avg       0.70      0.52      0.58       133


        p_VAGE  p_BEFORE  p_AFTER  p_SIMU
VAGUE       52         7        2       1
BEFORE      25        16        2       0
AFTER       15         3        1       0
SIMU         6         3        0       0
